# Cookbook

Here we provide a set of recipes detailing the functionality available in our set of packages

## Including the libraries

In [1]:
from cosmo import cosmo #to set the cosmology
from code_options import code_parameters #to set the cosmological parameters
from MTPK_estimate_beta import * #to estimate the spectra

## Setting the cosmology

The cosmological parameters have default values according to [Planck 2018](https://arxiv.org/abs/1807.06209). You can check the individual values just using

In [2]:
#Printing the default cosmology
cosmo().cosmo_print()

h = 0.678
Omega0_b = 0.048206
Omega0_cdm = 0.2589
Omega0_m = 0.307106
Omega0_k = 0.0
Omega0_DE = 0.692894
A_s = 2.1867466842075255e-09
ln10e10AsA = 3.085
n_s = 0.96
w0 = -1.0
w1 = 0.0
z_re = 9.99999
flat = True
gamma = 0.5454
matgrowcentral = 1e-05
zcentral = 1.0
c_light = 299792.458


''

The first argument of all the functions in this suite uses the set of cosmological parameters, then, set the cosmology is the first thing that you have to do in order to compute the power spectra.

### Setting the cosmology as the fiducial one

In [3]:
my_cosmology = cosmo()
my_cosmology.cosmo_print()

h = 0.678
Omega0_b = 0.048206
Omega0_cdm = 0.2589
Omega0_m = 0.307106
Omega0_k = 0.0
Omega0_DE = 0.692894
A_s = 2.1867466842075255e-09
ln10e10AsA = 3.085
n_s = 0.96
w0 = -1.0
w1 = 0.0
z_re = 9.99999
flat = True
gamma = 0.5454
matgrowcentral = 1e-05
zcentral = 1.0
c_light = 299792.458


''

### Setting another cosmology

Otherwise you can set different cosmology changing the parameters in the method `cosmo` according to:

In [4]:
my_cosmology = cosmo(h = 0.678, n_s = 0.96)
my_cosmology.cosmo_print()

h = 0.678
Omega0_b = 0.048206
Omega0_cdm = 0.2589
Omega0_m = 0.307106
Omega0_k = 0.0
Omega0_DE = 0.692894
A_s = 2.1867466842075255e-09
ln10e10AsA = 3.085
n_s = 0.96
w0 = -1.0
w1 = 0.0
z_re = 9.99999
flat = True
gamma = 0.5454
matgrowcentral = 1e-05
zcentral = 1.0
c_light = 299792.458


''

## Setting the code options

The second argument of all the functions in this suite uses the set of code options. These options define exactly what kind of data you are going to compute the power spectra and which are the spectra that you are obtaining. The fiducial option is given by:

In [5]:
#Defining the code options
my_code_options = code_parameters()
my_code_options.parameters_print()

method = both
mas_method = CIC
use_kdip_phys = False
kdip_phys = 0.005
multipoles_order = 4
use_padding = False
padding_length = [10, 10, 10]
use_theory_spectrum = False
theory_spectrum_file = theory_spectrum_file.txt
use_mask = False
mask_filename = mask.hdf5
mass_fun = [0.0156  0.00443 0.00143]
halo_bias = [1.572 1.906 2.442]
nhalos = 3
n_maps = 3
cell_size = 1.0
n_x = 128
n_y = 128
n_z = 128
n_x_orig = -64.0
n_y_orig = -64.0
n_z_orig = 10000.0
sel_fun_data = False
sel_fun_file = sel_fun-N128_halos.hdf5
kmin_bias = 0.05
kmax_bias = 0.15
kph_central = 0.1
dkph_bin = 0.01
use_kmin_phys = False
kmin_phys = 0.05
use_kmax_phys = True
kmax_phys = 1.0
whichspec = 1
use_cell_low_count_thresh = False
cell_low_count_thresh = 0.0
mult_sel_fun = 1.0
shift_sel_fun = 0.0
k_min_CAMB = 0.0001
k_max_CAMB = 1.0


''

### Setting different options

You can define different code options. However, some parameters are inherent from the dataset that you are using, so, pay attentio on that:

In [6]:
#Defining the code options
my_code_options = code_parameters(method = 'both', multipoles_order = 4)
my_code_options.parameters_print()

method = both
mas_method = CIC
use_kdip_phys = False
kdip_phys = 0.005
multipoles_order = 4
use_padding = False
padding_length = [10, 10, 10]
use_theory_spectrum = False
theory_spectrum_file = theory_spectrum_file.txt
use_mask = False
mask_filename = mask.hdf5
mass_fun = [0.0156  0.00443 0.00143]
halo_bias = [1.572 1.906 2.442]
nhalos = 3
n_maps = 3
cell_size = 1.0
n_x = 128
n_y = 128
n_z = 128
n_x_orig = -64.0
n_y_orig = -64.0
n_z_orig = 10000.0
sel_fun_data = False
sel_fun_file = sel_fun-N128_halos.hdf5
kmin_bias = 0.05
kmax_bias = 0.15
kph_central = 0.1
dkph_bin = 0.01
use_kmin_phys = False
kmin_phys = 0.05
use_kmax_phys = True
kmax_phys = 1.0
whichspec = 1
use_cell_low_count_thresh = False
cell_low_count_thresh = 0.0
mult_sel_fun = 1.0
shift_sel_fun = 0.0
k_min_CAMB = 0.0001
k_max_CAMB = 1.0


''

## Estimating the spectra

You can estimate the spectra using the function inside `MTPK_estimate_beta` code. The `handle_data` option follows for the name of the file where you are going to compute the spectra:

In [7]:
MTPK_estimate(my_cosmology, my_code_options, handle_data = "ExSHalos")



This is the Multi-tracer power spectrum estimator

Handle of this run (fiducial spectra, biases, etc.):  ExSHalos

Beggining CAMB calculations

Computing matter power spectrum for given cosmology...

0
CALLING CAMB NOW - HALOFIT
Time elapsed: 0.7513010501861572
.
Generating the k-space Grid...
.


/usr/local/lib/python3.8/dist-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


Will use the N = 3  simulation-only maps contained in directory /home/natalidesanti/doutorado/MTPK_github/MTPK/maps/sims/ExSHalos
.
Geometry: (nx,ny,nz) = (128,128,128),  cell_size=1.0 h^-1 Mpc
Geometry including bounding box: (nx,ny,nz) = (128,128,128)
.
Using power spectrum from CAMB + HaloFit
.
----------------------------------
.
Will estimate modes up to k[h/Mpc] =  1.0000  in bins with Delta_k = 0.0234
.
----------------------------------
.

----------------------------------

Central physical k values where spectra will be estimated: 0.1
Initializing the k-binning matrix...
Done with k-binning matrices. Time cost:  0.34 s
Memory occupied by the binning matrix:  16727
Originally k_bar was defined as: ['0.0441', '0.0516', '0.0590']
The true mean of k for each bin is: ['0.0444', '0.0513', '0.0587']

----------------------------------

Now estimating the power spectra...
Starting power spectra estimation
Initializing multi-tracer estimation toolbox...
Initializing traditional (FKP) 

0